In [1]:
#####Gaussian Process Logistic Regression model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.matlib
import scipy.linalg
import itertools
import calendar
from datetime import datetime
from datetime import timedelta
from scipy import sparse
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

In [2]:
##乱数を生成する関数を設定
#多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

#切断ポアソン分布を生成する関数
def rtpois(mu, a, b, n):
    FA = scipy.stats.poisson.cdf(a, mu)
    FB = scipy.stats.poisson.cdf(b, mu)
    return np.array(scipy.stats.poisson.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu), dtype="int")

#多変量正規分布の乱数を生成する関数
def rmvnorm(mu, Cov, hh, k):
    s = mu + np.random.normal(0, 1, hh*k).reshape(hh, k)
    P = np.linalg.cholesky(Cov)
    y = np.dot(P, s.T).T
    return y

#任意の相関行列を作る関数
def CorM(col, lower, upper, eigen_lower, eigen_upper, pattern):
    #相関行列の初期値を定義する
    if pattern==1:
        Prob = np.abs(lower) / (np.abs(lower) + np.abs(upper))
        z = np.random.binomial(1, Prob, col*col); z[z==0] = -1
        cov_vec = np.random.beta(1.0, 3.0, col*col) * z   #相関係数の乱数ベクトルを作成
    else:
        cov_vec = np.random.uniform(lower, upper, col*col)   #相関係数の乱数ベクトルを作成
    rho = np.tril(cov_vec.reshape(col, col), k=-1)   #乱数ベクトルを下三角行列化
    Sigma = (rho + rho.T) + np.diag(np.repeat(1, col))   #対角成分を1にする
    
    #相関行列を正定値行列に変更
    #固有値分解を実行
    eigen = np.linalg.eigh(Sigma)
    eigen_val = eigen[0] 
    eigen_vec = eigen[1]
    
    #固有値が負の数値を正にする
    for i in range(col):
        if eigen_val[i] < 0:
            eigen_val[i] = np.random.uniform(eigen_lower, eigen_upper, 1)
            
    #新しい相関行列の定義と対角成分を1にする
    Sigma = np.dot(np.dot(eigen_vec, np.diag(eigen_val)), eigen_vec.T)
    normalization_factor = np.dot(np.power(np.diag(Sigma), 0.5)[:, np.newaxis], np.power(np.diag(Sigma), 0.5)[np.newaxis, :])
    Cor = Sigma / normalization_factor
    return Cor

#相関行列から分散共分散行列に変換する関数
def covmatrix(Cor, sigma_lower, sigma_upper):
    sigma = (sigma_upper - sigma_lower) * rand(np.diag(Cor).shape[0]) + sigma_lower
    sigma_factor = np.dot(sigma[:, np.newaxis], sigma[np.newaxis, :])
    Cov = Cor * sigma_factor
    return Cov

#分散共分散行列から相関行列に変換する関数
def cov2cor(Cov):
    D = np.diag(np.power(np.diag(Cov), -1/2))
    corr = np.dot(np.dot(D, Cov), D)
    return corr

In [3]:
####データの生成####
##データの設定
hh = 3000
Lambda = np.random.gamma(10.0, 1/1.0, hh)
pt = np.random.poisson(Lambda, hh)
pt[pt < 2] =2
hhpt = np.sum(pt)

In [4]:
##idとインデックスを作成
#idの作成
d_id = np.repeat(np.arange(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスの設定
d_list = [i for i in range(hh)]
for i in range(hh):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")

In [5]:
##応答変数を生成
#カーネル関数を設定
K = CorM(hh, 0.1, 0.95, 0.1, 1.0, 0)
inv_K = np.linalg.inv(K)

#モデルパラメータを生成
alpha = np.repeat(0.0, hh)
beta = np.array([-0.5])
theta = np.random.multivariate_normal(alpha, K, 1).reshape(-1)
betat = beta.copy(); thetat = theta.copy()

#ベルヌーイ分布から二値変数を生成
mu = beta + theta[d_id]
Prob = np.exp(mu) / (1 + np.exp(mu))
y = np.random.binomial(1, Prob, hhpt)

In [6]:
####GP Logistic Regressionを推定####
##GP Logistic Regressionを推定するための関数
#多変量正規分布の条件付き期待値と分散を計算する関数
def cdMVN(mu, Cov, department, U):

    #分散共分散行列のブロック行列を定義
    index = np.delete(np.arange(Cov.shape[0]), department)
    Cov11 = Cov[department, ][:, department]
    Cov12 = Cov[department, ][:, index]
    Cov21 = Cov[:, department][index, ]
    Cov22 = Cov[index, ][:, index]

    #条件付き分散と条件付き平均を計算
    inv_Cov22 = np.linalg.inv(Cov22)
    CDinv = np.dot(Cov12, inv_Cov22)
    CDmu = mu[:, department] + np.dot(CDinv, (U[:, index] - mu[:, index]).T).T   #条件付き平均
    CDvar = Cov11 - np.dot(np.dot(Cov12, inv_Cov22), Cov21)   #条件付き分散
    return CDmu, CDvar

#対数事後分布を計算する関数
def Posterior(theta, beta, y, alpha, tau, inv_K, d_id, pattern):
    #ロジットモデルの対数尤度
    logit_exp = np.exp(beta + theta[d_id])
    Prob = logit_exp / (1 + logit_exp)
    LLi = y*np.log(Prob) + (1-y)*np.log(1-Prob)   #ロジットモデルの対数尤度   

    #多変量正規分布の対数事前分布
    if pattern==0:
        Prior = -beta / tau[0]
    elif  pattern==1:
        er = theta - alpha
        Prior = -1/2 * 1/tau[1] * np.dot(np.dot(er, inv_K), er)

    #対数事後分布の和
    Posterior = np.sum(LLi) + Prior
    return Posterior

#対数事後分布の微分関数
def dloglike(theta, beta, y, alpha, tau, inv_K, d_id, d_list, pattern):
    
    #ロジットの勾配ベクトルを定義
    logit_exp = np.exp(beta + theta[d_id])
    Prob = logit_exp / (1 + logit_exp)
    dlogit = y - Prob   #ロジットモデルの対数尤度の微分関数
    
    #対数事前分布の勾配ベクトルを定義
    if pattern==0:
        dnorm = -beta / tau[pattern]
    elif pattern==1:
        er = theta - alpha 
        dmvn = -1/tau[pattern] * np.dot(inv_K, er)   #多変量正規分布の対数事前分布の微分関数

    #勾配ベクトルの和
    if pattern==0:
        LLd = -(np.sum(dlogit) + dnorm)
    elif pattern==1:
        dlogit_sums = np.repeat(0.0, hh)
        for i in range(hh):
            dlogit_sums[i] = np.sum(dlogit[d_list[i]], axis=0)
        LLd = -(dlogit_sums + dmvn)
    return LLd

#期待値パラメータのリープフロッグ法を解く関数
def leapfrog_mu(r, z, D, e, L): 
    def leapfrog_step(r, z, e):
        r2 = r - e * D(theta, z, y, alpha, tau, inv_K, d_id, d_list, 0) / 2
        z2 = z + e * r2
        r2 = r2 - e * D(theta, z2, y, alpha, tau, inv_K, d_id, d_list, 0) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result

#GPモデルのリープフロッグ法を解く関数
def leapfrog_gp(r, z, D, e, L): 
    def leapfrog_step(r, z, e):
        r2 = r - e * D(z, beta, y, alpha, tau, inv_K, d_id, d_list, 1) / 2
        z2 = z + e * r2
        r2 = r2 - e * D(z2, beta, y, alpha, tau, inv_K, d_id, d_list, 1) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result

In [7]:
##アルゴリズムの設定
k = 10
R = 2000
keep = 2
burnin = int(500/keep)
iter = 0
disp = 50
e1 = 0.01
e2 = 0.01
L = 5

In [8]:
##初期値の設定
#事前分布の設定
tau = np.array([100.0, 2.5])
inv_K = np.linalg.inv(K)

#モデルパラメータの初期値
beta = np.array([0.0])
theta = np.random.normal(0, 1.0, hh)

#モデルの期待値
mu = beta + theta[d_id]
Prob = np.exp(mu) / (1 + np.exp(mu))

In [9]:
#パラメータの保存用配列
BETA = np.repeat(0.0, int(R/keep))
THETA = np.zeros((int(R/keep), hh))

In [10]:
##対数尤度の基準値
#真値での対数尤度
LLbest = float(Posterior(thetat, betat, y, alpha, tau, inv_K, d_id, 0))
print(LLbest)

#1パラメータモデルの対数尤度
beta0 = np.array([np.log(np.mean(y) / np.mean(np.mean(1-y)))])
theta0 = np.repeat(0.0, hh)
LLst = float(Posterior(theta0, beta0, y, alpha, tau, inv_K, d_id, 0))
print(LLst)

-17784.922145746394
-20343.53634890514


In [15]:
####HMCでパラメータをサンプリング####
for rp in range(R):
    
    ##モデルの期待値をサンプリング
    #HMCの新しいパラメータを生成
    rold = np.random.normal(0, 1.0, 1)
    betad = beta.copy()

    #リープフロッグ法による1ステップ移動
    res = leapfrog_mu(rold, betad, dloglike, e1, L)
    rnew = res[0]
    betan = res[1]

    #移動前と移動後のハミルトニアン
    Hnew = -Posterior(theta, betan, y, alpha, tau, inv_K, d_id, 0) + np.power(rnew, 2)/2
    Hold = -Posterior(theta, betad, y, alpha, tau, inv_K, d_id, 0) + np.power(rold, 2)/2

    #新しいパラメータを採択
    rand = np.random.uniform(0, 1, 1)
    gamma1 = np.min(np.append(1, np.exp(Hold - Hnew)))
    flag = np.array((gamma1 > rand), dtype="int")
    beta = flag*betan + (1-flag)*betad


    ##ガウス過程のパラメータをサンプリング
    #HMCの新しいパラメータを生成
    rold = np.random.normal(0, 1.0, hh)
    thetad = theta.copy()

    #リープフロッグ法による1ステップ移動
    res = leapfrog_gp(rold, thetad, dloglike, e2, L)
    rnew = res[0]
    thetan = res[1]

    #移動前と移動後のハミルトニアン
    Hnew = -Posterior(thetan, beta, y, alpha, tau, inv_K, d_id, 1) + np.sum(np.power(rnew, 2))/2
    Hold = -Posterior(thetad, beta, y, alpha, tau, inv_K, d_id, 1) + np.sum(np.power(rold, 2))/2

    #新しいパラメータを採択
    rand = np.random.uniform(0, 1, 1)
    gamma2 = np.min(np.append(1, np.exp(Hold - Hnew)))
    flag = np.array((gamma2 > rand), dtype="int")
    theta = flag*thetan + (1-flag)*thetad
    
    
    ##サンプリング結果の格納と表示
    if rp%keep==0:
        mkeep = int(rp/keep)
        BETA[mkeep] = beta
        THETA[mkeep, ] = theta
        
    #サンプリング結果の格納
    if rp%disp==0:
        #対数尤度を更新
        LL = float(Posterior(theta, beta, y, alpha, tau, inv_K, d_id, 0))
        
        #サンプリング結果を表示
        print(rp)
        print(np.round([gamma1, gamma2], 3))
        print(np.round([LL, LLst, LLbest], 1))

0
[0.705 0.995]
[-17632.9 -20343.5 -17784.9]
50
[1.    0.997]
[-17628.  -20343.5 -17784.9]
100
[1. 1.]
[-17640.6 -20343.5 -17784.9]
150
[0.983 1.   ]
[-17642.3 -20343.5 -17784.9]
200
[0.908 0.993]
[-17611.8 -20343.5 -17784.9]
250
[0.998 1.   ]
[-17607.8 -20343.5 -17784.9]
300
[1. 1.]
[-17588.8 -20343.5 -17784.9]
350
[0.924 0.997]
[-17577.5 -20343.5 -17784.9]
400
[0.833 1.   ]
[-17568.6 -20343.5 -17784.9]
450
[0.909 1.   ]
[-17561.1 -20343.5 -17784.9]
500
[1. 1.]
[-17573.4 -20343.5 -17784.9]
550
[1.    0.997]
[-17590.8 -20343.5 -17784.9]
600
[0.997 1.   ]
[-17617.1 -20343.5 -17784.9]
650
[1.    0.998]
[-17601.6 -20343.5 -17784.9]
700
[1.    0.998]
[-17588.4 -20343.5 -17784.9]
750
[1. 1.]
[-17567.  -20343.5 -17784.9]
800
[0.88  0.994]
[-17576.2 -20343.5 -17784.9]
850
[1. 1.]
[-17569.6 -20343.5 -17784.9]
900
[0.854 0.995]
[-17600.9 -20343.5 -17784.9]
950
[0.977 1.   ]
[-17552.4 -20343.5 -17784.9]
1000
[1. 1.]
[-17555.7 -20343.5 -17784.9]
1050
[0.945 1.   ]
[-17552.4 -20343.5 -17784.9]
110

In [18]:
rs = np.arange(burnin, int(R/keep))
np.mean(THETA[rs, ], axis=0)

array([-0.2435077 ,  0.93284731,  1.89459269, ...,  1.55705272,
        0.70606681,  2.6917227 ])

In [19]:
thetat

array([-0.58194592, -0.04039134,  1.28729717, ...,  1.9570513 ,
        0.29472035,  2.10612741])